# Setup

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!nvidia-smi
%pip install ultralytics
import ultralytics
ultralytics.checks()
import os
HOME = os.getcwd()
print("HOME:", HOME)
%pip install -q 'git+https://github.com/facebookresearch/segment-anything.git'
%pip install -q jupyter_bbox_widget roboflow dataclasses-json supervision
!mkdir -p {HOME}/weights
!wget -q https://dl.fbaipublicfiles.com/segment_anything/sam_vit_h_4b8939.pth -P {HOME}/weights
CHECKPOINT_PATH = os.path.join(HOME, "weights", "sam_vit_h_4b8939.pth")
print(CHECKPOINT_PATH, "; exist:", os.path.isfile(CHECKPOINT_PATH))
import torch
DEVICE = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
MODEL_TYPE = "vit_h"

Ultralytics YOLOv8.2.41 🚀 Python-3.10.12 torch-2.3.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 32.6/78.2 GB disk)
HOME: /content
  Preparing metadata (setup.py) ... done
/content/weights/sam_vit_h_4b8939.pth ; exist: True


# Imports

In [ ]:
from segment_anything import sam_model_registry, SamAutomaticMaskGenerator, SamPredictor
from ultralytics import YOLO
import numpy as np

#YOLOv8 bounding box extraction

In [ ]:
IMAGE_PATH ='/content/drive/MyDrive/image dataset/wine glass_from_front_0.png'
desired_label=["remote","bottle","cup","bowl","vase","scissors","tooth brush","wine glass","baseball bat"]

In [ ]:
def process_image(IMAGE_PATH,desired_label):
  model = YOLO("yolov8n.pt")
  results = model([IMAGE_PATH])
  for result in results:
    boxes = result.boxes
    for i in range(boxes.shape[0]):
      if (model.names[int(boxes.cls[i].item())] in desired_label):
        bounding_box=np.array([(boxes.xyxy[i].cpu().numpy()[0]),
                      (boxes.xyxy[i].cpu().numpy()[1]),
                      (boxes.xyxy[i].cpu().numpy()[2]),
                      (boxes.xyxy[i].cpu().numpy()[3])])
        return bounding_box


In [ ]:
bbox=process_image(IMAGE_PATH,desired_label)


0: 640x640 1 wine glass, 9.9ms
Speed: 3.4ms preprocess, 9.9ms inference, 1.7ms postprocess per image at shape (1, 3, 640, 640)
wine glass


In [ ]:
bbox

array([     275.18,      51.227,      739.29,       992.5], dtype=float32)

#SAM segmentation

In [ ]:
sam = sam_model_registry[MODEL_TYPE](checkpoint=CHECKPOINT_PATH).to(device=DEVICE)

In [ ]:
mask_generator = SamAutomaticMaskGenerator(sam)

In [ ]:
mask_predictor = SamPredictor(sam)

In [ ]:
import cv2
import numpy as np
import supervision as sv

In [ ]:
image_bgr = cv2.imread(IMAGE_PATH)
image_rgb = cv2.cvtColor(image_bgr, cv2.COLOR_BGR2RGB)
def sam_segment(bbox,image_rgb):
  mask_predictor.set_image(image_rgb)

  masks, scores, logits = mask_predictor.predict(
      box=bbox,
      multimask_output=False
  )
  detections = sv.Detections(
    xyxy=sv.mask_to_xyxy(masks=masks),
    mask=masks
  )
  box_annotator = sv.BoxAnnotator(color=sv.Color.red())
  mask_annotator = sv.MaskAnnotator(color=sv.Color.red(), color_lookup=sv.ColorLookup.INDEX)

  detections = sv.Detections(
      xyxy=sv.mask_to_xyxy(masks=masks),
      mask=masks
  )
  detections = detections[detections.area == np.max(detections.area)]

  source_image = box_annotator.annotate(scene=image_bgr.copy(), detections=detections, skip_label=True)
  segmented_image = mask_annotator.annotate(scene=image_bgr.copy(), detections=detections)
  return detections

In [ ]:
detections=sam_segment(bbox,image_rgb)

SupervisionWarnings: red is deprecated: `Color.red()` is deprecated and will be removed in `supervision-0.22.0`. Use `Color.RED` instead.
SupervisionWarnings: red is deprecated: `Color.red()` is deprecated and will be removed in `supervision-0.22.0`. Use `Color.RED` instead.
SupervisionWarnings: annotate is deprecated: `BoxAnnotator` is deprecated and will be removed in `supervision-0.22.0`. Use `BoundingBoxAnnotator` and `LabelAnnotator` instead


In [ ]:
import cv2 as cv
import random
from math import atan2, cos, sin, sqrt, pi , tan , radians
from shapely.geometry import LineString,Polygon
from google.colab.patches import cv2_imshow
def calculate_angle_and_coordinates(ann,image,no_of_objects,bbox):
      bw=ann.mask[0].astype(int)
      bw = np.array(bw, np.uint8)
      contours, _ = cv.findContours(bw, cv.RETR_LIST, cv.CHAIN_APPROX_NONE)
      area_max=0
      for k, c in enumerate(contours):
          area = cv.contourArea(c)
          if area>area_max:
            area_max=area
            c_max=c
      i=random.randint(1,c_max.shape[0])
      j=random.randint(1,c_max.shape[0])
      cv.circle(image,(int(c_max[i][0][0]),int(c_max[i][0][1])),20,(0,255,0),-1)
      cv.circle(image,(int(c_max[j][0][0]),int(c_max[j][0][1])),20,(0,255,0),-1)
      cv2_imshow(image)